In [1]:
import cPickle as pickle
import gzip
import numpy
from midi_to_statematrix import *

import multi_training
import model


def gen_adaptive(m, pcs, times, keep_thoughts=False, name="final"):
    xIpt, xOpt = map(lambda x: numpy.array(x, dtype='int8'), multi_training.getPieceSegment(pcs))
    all_outputs = [xOpt[0]]
    if keep_thoughts:
        all_thoughts = []
    m.start_slow_walk(xIpt[0])
    cons = 1
    for time in range(multi_training.batch_len * times):
        resdata = m.slow_walk_fun(cons)
        nnotes = numpy.sum(resdata[-1][:, 0])
        if nnotes < 2:
            if cons > 1:
                cons = 1
            cons -= 0.02
        else:
            cons += (1 - cons) * 0.3
        all_outputs.append(resdata[-1])
        if keep_thoughts:
            all_thoughts.append(resdata)
    noteStateMatrixToMidi(numpy.array(all_outputs), 'output/' + name)
    if keep_thoughts:
        pickle.dump(all_thoughts, open('output/' + name + '.p', 'wb'))


def fetch_train_thoughts(m, pcs, batches, name="trainthoughts"):
    all_thoughts = []
    for i in range(batches):
        ipt, opt = multi_training.getPieceBatch(pcs)
        thoughts = m.update_thought_fun(ipt, opt)
        all_thoughts.append((ipt, opt, thoughts))
    pickle.dump(all_thoughts, open('output/' + name + '.p', 'wb'))

Using gpu device 0: GeForce GTX 860M (CNMeM is enabled with initial size: 90.0% of memory, CuDNN 4004)


In [2]:
pieces = multi_training.loadPieces('music/pieces.pkl')

Loaded preprocessed notes...


In [3]:
m = model.Model([300, 300], [100, 50], dropout=0.5)

model.py:367: UserWarning: Updating an `OrderedUpdates` with a non-ordered dictionary with 2+ elements could make your code non-deterministic
  self.walk_input: next_input
model.py:370: UserWarning: Updating an `OrderedUpdates` with a non-ordered dictionary with 2+ elements could make your code non-deterministic
  updates.update({hidden:newstate for hidden, newstate, layer in zip(self.walk_hiddens, new_states, self.time_model.layers) if has_hidden(layer)})


In [5]:
m.learned_config = pickle.load(open("output/final_learned_config_8000.p", "rb"))

In [8]:
batches = 7000

multi_training.trainPiece(m, pieces, batches)#, notes_to_input = None)

pickle.dump(m.learned_config, open("output/final_learned_config_{0}.p".format(batches), "wb"))

MemoryError: Error allocating 237744000 bytes of device memory (CNMEM_STATUS_OUT_OF_MEMORY).
Apply node that caused the error: GpuAlloc{memset_0=True}(CudaNdarrayConstant{[[[ 0.]]]}, Elemwise{Composite{(Switch(i0, Switch(LT((i1 + (i2 - i3)), i3), i3, (i1 + (i2 - i3))), Switch(LT(i1, (i2 - i3)), i1, (i2 - i3))) - i3)}}.0, Elemwise{switch,no_inplace}.0, Elemwise{Composite{Switch(EQ(i0, i1), i2, i0)}}.0)
Toposort index: 375
Inputs types: [CudaNdarrayType(float32, (True, True, True)), TensorType(int64, scalar), TensorType(int64, scalar), TensorType(int64, scalar)]
Inputs shapes: [(1, 1, 1), (), (), ()]
Inputs strides: [(0, 0, 0), (), (), ()]
Inputs values: [CudaNdarray([[[ 0.]]]), array(127), array(780), array(600)]
Outputs clients: [[forall_inplace,gpu,grad_of_scan_fn}(Elemwise{maximum,no_inplace}.0, GpuElemwise{mul,no_inplace}.0, GpuAlloc{memset_0=True}.0, GpuAlloc{memset_0=True}.0, GpuAlloc{memset_0=True}.0, Subtensor{int64:int64:int64}.0, GpuSubtensor{int64:int64:int64}.0, GpuSubtensor

In [4]:
gen_adaptive(m,pcs,10,name="composition_{0}".format(batches))

In [5]:
p = pickle.load(open('music/notes_to_input.pkl', "rb"))

In [5]:
import numpy
from data import *
notes_to_input = numpy.array([noteStateMatrixToInputForm(p) for p in pieces.values()])